In [49]:
import pandas as pd
import matplotlib.pyplot as plt
import calendar
from datetime import datetime 

df = pd.read_csv("5 - Muesli Project raw data - Orders.csv", header = 1)
#functions
#get weekday
def weekday(date):  
    return calendar.day_name[date.weekday()] 

#Adding days to date
def add_days_to_date(date,days):
    return date + datetime.timedelta(days=days)

df = pd.read_csv("5 - Muesli Project raw data - Orders.csv", header = 1)


In [50]:
df = pd.read_csv("5 - Muesli Project raw data - Orders.csv", header = 1)
warehouse_workdays = ["Monday", "Tuesday","Wednesday","Thursday","Friday"]
transport_workdays = ["Monday","Wednesday","Friday"]
warehousestorage_time = 1
warehouse_prep_time = 2
express = warehousestorage_time
no_express = warehousestorage_time + warehouse_prep_time

In [51]:
#adjust column names
df.columns = df.columns.str.replace(" ","_")
df.columns = map(str.lower, df.columns)

In [52]:
#order date into date
df["order_date"] = df["order_date"].apply(lambda date: datetime.strptime(str(date), "%d/%m/%Y"))

In [53]:
df.shape

(9994, 19)

In [54]:
df.columns

Index(['index', 'order_id', 'order_date', 'ship_mode', 'customer_id',
       'customer_name', 'origin_channel', 'country/region', 'city', 'state',
       'postal_code', 'region', 'category', 'sub-category', 'product_id',
       'sales', 'quantity', 'discount', 'profit'],
      dtype='object')

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   index           9994 non-null   int64         
 1   order_id        9994 non-null   object        
 2   order_date      9994 non-null   datetime64[ns]
 3   ship_mode       9994 non-null   object        
 4   customer_id     9994 non-null   object        
 5   customer_name   9994 non-null   object        
 6   origin_channel  9994 non-null   object        
 7   country/region  9994 non-null   object        
 8   city            9994 non-null   object        
 9   state           9994 non-null   object        
 10  postal_code     9983 non-null   float64       
 11  region          9994 non-null   object        
 12  category        9994 non-null   object        
 13  sub-category    9994 non-null   object        
 14  product_id      9994 non-null   object        
 15  sale

In [56]:
#create column: Weekdays of order_date
df["order_date_weekday"] = df["order_date"].apply(lambda date: weekday(date))

In [57]:
#check
df.order_date_weekday.unique()

array(['Wednesday', 'Monday', 'Thursday', 'Saturday', 'Tuesday', 'Friday',
       'Sunday'], dtype=object)

In [58]:
#delivery day: adding days whether express or not
import datetime
df["order_express_considered"] = df.apply(lambda x: add_days_to_date(x["order_date"],express) if x["ship_mode"] == "First Class" else add_days_to_date(x["order_date"],no_express), axis = 1 )

In [59]:
#delivery day: adding days whether order date is on warehouse workdays
df["order_leaving_warehouse"] = df.apply(lambda x: add_days_to_date(x["order_express_considered"],days =2) if x["order_date_weekday"] == "Saturday" else add_days_to_date(x["order_express_considered"],days =1) if x["order_date_weekday"] == "Sunday" else x["order_express_considered"], axis = 1)

In [60]:
#adding new weekday column for "order_leaving_warehouse"
df["order_leaving_warehouse_weekday"] = df["order_leaving_warehouse"].apply(lambda date: weekday(date))

In [70]:
#delivery day: adding days considering logistics
df["delivery_day"] = df.apply(lambda x: add_days_to_date(x["order_leaving_warehouse"],days =5) if x["order_leaving_warehouse_weekday"] == "Wednesday" else add_days_to_date(x["order_leaving_warehouse"],days =3) if x["order_leaving_warehouse_weekday"] == "Monday" else add_days_to_date(x["order_leaving_warehouse"],days =6) if x["order_leaving_warehouse_weekday"] == "Tuesday" else add_days_to_date(x["order_leaving_warehouse"],days =4) if x["order_leaving_warehouse_weekday"] == "Thursday" else add_days_to_date(x["order_leaving_warehouse"],days =3) if x["order_leaving_warehouse_weekday"] == "Friday" else add_days_to_date(x["order_leaving_warehouse"],days =5) if x["order_leaving_warehouse_weekday"] == "Saturday" else add_days_to_date(x["order_leaving_warehouse"],days =4) if x["order_leaving_warehouse_weekday"] == "Sunday" else "", axis = 1)

In [71]:
#delivery_time calculation
df["delivery_time[days]"] = df.apply(lambda x : x["delivery_day"] - x["order_date"], axis = 1)

In [61]:
#delivery day: adding days considering logistics
df["delivery_day"] = df.apply(lambda x: add_days_to_date(x["order_leaving_warehouse"],days =5) if x["order_leaving_warehouse_weekday"] == "Wednesday" else add_days_to_date(x["order_express_considered"],days =1) if x["order_date_weekday"] == "Sunday" else x["order_express_considered"], axis = 1)

In [62]:
#check
df.ship_mode.unique()

array(['Second Class', 'First Class', 'Standard Class'], dtype=object)

In [77]:
#check
df.head(10)

,index,order_id,order_date,ship_mode,customer_id,customer_name,origin_channel,country/region,city,state,...,sales,quantity,discount,profit,order_date_weekday,order_express_considered,order_leaving_warehouse,order_leaving_warehouse_weekday,delivery_day,delivery_time[days]
0,27,CA-2019-121755,2019-01-16,Second Class,EH-13945,Eric Hoffmann,Email,United States,Los Angeles,California,...,90.570,3,0.0,11.7741,Wednesday,2019-01-19,2019-01-19,Saturday,2019-01-24,8 days
1,45,CA-2019-118255,2019-03-11,First Class,ON-18715,Odella Nelson,Sales,United States,Eagan,Minnesota,...,45.980,2,0.0,19.7714,Monday,2019-03-12,2019-03-12,Tuesday,2019-03-18,7 days
2,48,CA-2019-169194,2019-06-20,Standard Class,LH-16900,Lena Hernandez,Email,United States,Dover,Delaware,...,45.000,3,0.0,4.9500,Thursday,2019-06-23,2019-06-23,Sunday,2019-06-27,7 days
3,60,CA-2019-111682,2019-06-17,First Class,TB-21055,Ted Butterfield,Email,United States,Troy,New York,...,30.000,2,0.0,3.3000,Monday,2019-06-18,2019-06-18,Tuesday,2019-06-24,7 days
4,63,CA-2018-135545,2018-11-24,Standard Class,KM-16720,Kunst Miller,Email,United States,Los Angeles,California,...,13.980,2,0.0,6.1512,Saturday,2018-11-27,2018-11-29,Thursday,2018-12-03,9 days
5,87,CA-2020-155558,2020-10-26,Standard Class,PG-18895,Paul Gonzalez,Email,United States,Rochester,Minnesota,...,19.990,1,0.0,6.7966,Monday,2020-10-29,2020-10-29,Thursday,2020-11-02,7 days
6,101,CA-2019-158568,2019-08-29,Standard Class,RB-19465,Rick Bensley,Facebook,United States,Chicago,Illinois,...,95.976,3,0.2,-10.7973,Thursday,2019-09-01,2019-09-01,Sunday,2019-09-05,7 days
7,104,US-2018-156867,2018-11-13,Standard Class,LC-16870,Lena Cacioppo,Email,United States,Aurora,Colorado,...,238.896,6,0.2,-26.8758,Tuesday,2018-11-16,2018-11-16,Friday,2018-11-19,6 days
8,107,CA-2020-119004,2020-11-23,Standard Class,JM-15250,Janet Martin,Email,United States,Charlotte,North Carolina,...,74.112,8,0.2,17.6016,Monday,2020-11-26,2020-11-26,Thursday,2020-11-30,7 days
9,110,CA-2018-129476,2018-10-15,Standard Class,PA-19060,Pete Armstrong,Facebook,United States,Orland Park,Illinois,...,339.960,5,0.2,67.9920,Monday,2018-10-18,2018-10-18,Thursday,2018-10-22,7 days


In [69]:
df.groupby('order_date_weekday').max()['profit'].reset_index()

,order_date_weekday,profit
0,Friday,3177.4750
1,Monday,6719.9808
2,Saturday,4630.4755
3,Sunday,1415.4296
4,Thursday,3919.9888
5,Tuesday,5039.9856
6,Wednesday,8399.9760


In [68]:
df.groupby('order_date_weekday').sum()['profit'].reset_index()

,order_date_weekday,profit
0,Friday,45209.9540
1,Monday,33875.3938
2,Saturday,27533.0636
3,Sunday,23283.3214
4,Thursday,51667.4876
5,Tuesday,59023.3644
6,Wednesday,45804.4369


In [75]:
df.groupby(['order_date_weekday', 'region']).sum()['profit'].sort_values().reset_index()

,order_date_weekday,region,profit
0,Sunday,Central,-1525.5089
1,Friday,Central,1346.1813
2,Monday,East,3485.6032
3,Monday,South,3965.5553
4,Saturday,South,4291.1141
5,Monday,Central,5299.0130
6,Sunday,South,5595.7997
7,Saturday,East,5603.6352
8,Wednesday,South,5978.8709
9,Thursday,South,6914.4550


In [80]:
df.groupby(['delivery_time[days]', 'order_date_weekday', 'ship_mode']).sum()['profit'].sort_values().reset_index()

,delivery_time[days],order_date_weekday,ship_mode,profit
0,8 days,Sunday,Second Class,3214.4112
1,9 days,Saturday,First Class,4416.8792
2,8 days,Wednesday,Second Class,5266.7355
3,8 days,Sunday,First Class,5352.0515
4,6 days,Friday,First Class,5736.7979
5,7 days,Monday,Standard Class,6984.1715
6,6 days,Tuesday,Second Class,7154.4530
7,5 days,Wednesday,First Class,7469.6365
8,9 days,Saturday,Second Class,7578.6762
9,6 days,Friday,Second Class,9343.8328
